In [1]:
from master_dac.rld import stk

In [2]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stk_actor.agent import UnifiedSACPolicy
from stk_actor.wrappers import PreprocessObservationWrapper
import torch

from pystk2_gymnasium import AgentSpec

vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", seed=0,
                        # wrapper_class=lambda x : ObsTimeExtensionWrapper(PreprocessObservationWrapper(x)), 
                        wrapper_class=lambda x : (PreprocessObservationWrapper(x)), 
                       n_envs=1, env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"),  #'track':'abyss', #'num_kart':2, 'difficulty':0
})

# net_arch=[512,512,512]
# activation_fn=torch.nn.SiLU
# filename = 'policy_512_512_512_SiLU_2_statedict'

# net_arch=[512,512,512,512,512]
net_arch=[1024,1024,1024]
activation_fn=torch.nn.Tanh
# filename = 'policy_512_512_512_512_SiLU_3_statedict'
# filename = 'airl_policy_512_512_512_512_SiLU_statedict'
# filename = 'policy_normed_512_512_512_512_SiLU_statedict'
# filename = 'policy_normed_128_128_SiLU_statedict'
filename = 'policy_normed_1024_1024_1024_Tanh_statedict'

model = PPO(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,), 
        device='cpu',
        n_steps=5,
        tensorboard_log="./outputs/"
)
action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename))
model.policy.load_state_dict(unified_policy.shared.state_dict())
# model.policy.load_state_dict(torch.load(filename))
vec_env.close()
# model = PPO.load("ppti_ppo_1500", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), })
# model = PPO.load("ppti_ppo2_1500_batch128_clip01_ent0001", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), })
mean = torch.load('buffer_mean', map_location='cpu')
std = torch.load('buffer_std', map_location='cpu')

..:: Antarctica Rendering Engine 2.0 ::..
Using cpu device


/opt/homebrew/Caskroom/miniforge/base/envs/bbrl/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 5
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=5 and n_envs=1)
  warnings.warn(
/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_8158/341950868.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be exec

In [3]:
mean.tolist()

[0.7800899147987366,
 0.0,
 -0.10567483305931091,
 0.04174956679344177,
 3.9957878589630127,
 -0.587861955165863,
 700.8651733398438,
 1.4090956449508667,
 7.841001934139058e-05,
 8.439883822575212e-05,
 0.7182344198226929,
 -0.013836018741130829,
 -0.01726570352911949,
 41.80649185180664,
 -0.016669513657689095,
 -0.0048374151811003685,
 50.54801940917969,
 -0.01393372006714344,
 0.0035251181107014418,
 60.80333709716797,
 -0.033544689416885376,
 -0.005445714108645916,
 82.7087631225586,
 -0.032481346279382706,
 -0.030109180137515068,
 92.82120513916016,
 -0.03025653399527073,
 -0.04940488189458847,
 35.81877899169922,
 -0.0316675528883934,
 -0.04317280277609825,
 35.191680908203125,
 -0.025999752804636955,
 -0.026596002280712128,
 22.43626594543457,
 -0.03653942421078682,
 -0.0246012844145298,
 26.38672637939453,
 -0.04061321169137955,
 -0.012653408572077751,
 31.20808982849121,
 0.07753840833902359,
 696.9466552734375,
 705.3189086914062,
 699.6309204101562,
 707.5324096679688,
 701

In [4]:
std.tolist()

[3.6490633487701416,
 0.0,
 1.6003646850585938,
 1.7796735763549805,
 12.503802299499512,
 13.113574028015137,
 490.5068664550781,
 1.2362719774246216,
 0.03838471323251724,
 0.030686920508742332,
 0.07883542776107788,
 0.7218800783157349,
 0.4027252495288849,
 34.89804458618164,
 0.7576894760131836,
 0.4520794749259949,
 36.360130310058594,
 0.8363945484161377,
 0.5438740253448486,
 42.65901184082031,
 1.014235258102417,
 0.7227504849433899,
 48.12322235107422,
 1.0808465480804443,
 0.8007359504699707,
 48.81171417236328,
 1.574799656867981,
 1.6711925268173218,
 28.26801872253418,
 1.5987789630889893,
 1.7467477321624756,
 35.90023422241211,
 1.3176236152648926,
 1.425809383392334,
 33.22978591918945,
 1.4683250188827515,
 1.625714898109436,
 38.57169723510742,
 1.572075366973877,
 1.7933919429779053,
 44.3121337890625,
 0.05287080258131027,
 490.9122009277344,
 490.0939636230469,
 489.74609375,
 488.879150390625,
 488.27923583984375,
 487.4639587402344,
 487.1041259765625,
 486.3286

In [ ]:
# 'fortmagma',
# 'ravenbridge_mansion',
# 'snowmountain',
# 'cocoa_temple',
# 'sandtrack',    
# 'scotland', 
# 'stk_enterprise',
# 'volcano_island', # 1104
# 'xr591', # 864
from stk_actor.wrappers import StuckStopWrapper


tracks = [
    # 'black_forest',
    'olivermath', # 244
    # 'minigolf', # 434
    # 'gran_paradiso_island', # 683
    # 'candela_city', # 521
    # 'mines', # 639
    # 'snowmountain', # 589
    # 'abyss', # 612
    # 'cornfield_crossing', # 638
    # 'hacienda', # 581
    # 'lighthouse', # 452
    # 'snowtuxpeak', # 482
    # 'zengarden', # 356


    # 'fortmagma',
    # 'ravenbridge_mansion',
    # 'snowmountain',
    # 'cocoa_temple',
    # 'sandtrack',    
    #'scotland', 
    # 'stk_enterprise',
    # 'volcano_island', # 1104
    # 'xr591', # 864        

]
for track in tracks:
# while True:
    vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", #seed=0,
                        # wrapper_class=lambda x : ObsTimeExtensionWrapper(PreprocessObservationWrapper(x)), 
                        wrapper_class=lambda x : (
                            StuckStopWrapper(
                                PreprocessObservationWrapper(x, ret_dict=False, norm=False),
                                n=26,
                            )
                        ), 
                       n_envs=1, env_kwargs={
    'render_mode':"human", 'agent':AgentSpec(use_ai=False, name="walid"), 'laps':1, 
    'track':track,
    'num_kart':6, 'difficulty':2
    })
    obs = vec_env.reset()
    dones = [False]
    ix = 0
    while (True and not any(dones)):
        
        obs = torch.tensor(obs)
        
        obs = (obs - mean) / (std + 1e-8)
        action, _states = model.predict(obs, deterministic=False)
        obs, rewards, dones, info = vec_env.step(action)
        vec_env.render("human")
        # print(dones)
        ix += 1
        if ix == 1500:
            break
    print(track, ix)
    vec_env.close()



        




..:: Antarctica Rendering Engine 2.0 ::..


2025-01-22 00:18:47.505 python[8244:236283] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-22 00:18:47.505 python[8244:236283] +[IMKInputSession subclass]: chose IMKInputSession_Modern


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]
[0.4, 0.4, 0.4, 0.4, 0.4,

In [6]:
# import torch
# torch.save( model.policy.state_dict(), 'ppo_policy_512_512_512_512_SiLU_4_statedict')

In [7]:
vec_env.envs[0]

<StuckStopWrapper<PreprocessObservationWrapper<Monitor<FlattenerWrapper<DiscreteActionsWrapper<PolarObservations<ConstantSizedObservations<TimeLimit<OrderEnforcing<PassiveEnvChecker<STKRaceEnv<supertuxkart/flattened_multidiscrete-v0>>>>>>>>>>>>